In [1]:
from collections import Counter
import os

In [52]:
class Folder(object): 
    
    def __init__(self, root, subject_substr=""): 
        
        if "\\" in root: 
            raise Exception("Please replace the backslashes in your path with forwardslashes.")
        self.root = root
        self.subfolders = self.get_subjects(subject_substr)
        
        self.walk_history = []
        self.walk_history.append(self.subfolders)

        self.walk_history_lost = []
        self.walk_history_lost.append([])
        
     
    def get_folders_files(self, path):
        for root, folders, files in os.walk(path):
            break   
        
        return folders, files
        
    def get_subjects(self, subject_substr):         
        subjects, _ = self.get_folders_files(self.root)
        
        return [s for s in subjects if subject_substr in s]

    def down(self, equals=[], contains=[], does_not_contain=[], print_absent=False): 
                    
        print(f"\nSubfolder equals {str(equals)},   contains {str(contains)},   does not contain {str(does_not_contain)}")
        subfolders_present = []
        subfolders_absent = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            
            folders, files = self.get_folders_files(path)
            
            for string in equals: 
                folders = [f for f in folders if string.lower() == f.lower()]
            
            for string in contains: 
                folders = [f for f in folders if string.lower() in f.lower()]
            
            for string in does_not_contain: 
                folders = [f for f in folders if string.lower() not in f.lower()]
            
            if len(folders) == 0: 
                subfolders_absent.append(subfolder)
                
            elif len(folders) == 1: 
                subfolders_present.append(os.path.join(subfolder, folders[0]))
                
            elif len(folders) > 1: 
                print(f"\n{subfolder} has multiple subfolders with the specified name:")
                print(folders)
                subfolders_present.append(os.path.join(subfolder, folders[0]))
        
        if len(subfolders_present) == 0:
            print("\nNo folders contained the specified subfolders, therefore this step is not executed.")
        
        else:    
            print(f"\n{len(subfolders_present)} of {len(self.subfolders)} folders contained specified subfolder.")
            if print_absent: 
                print("\nFolders without subfolder")
                for s in subfolders_absent:
                    print("\t", s)

            self.walk_history.append(subfolders_present)
            self.walk_history_lost.append(subfolders_absent)
            self.subfolders = subfolders_present
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
        
            
    def up(self): 
        if len(self.walk_history) == 1:
            print("You can not go up as you are already at the subject folder")
        
        else: 
            self.subfolders = self.walk_history[-2]
            self.walk_history = self.walk_history[:-1]
            self.walk_history_lost = self.walk_history_lost[:-1]  
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
    
    def sample_down(self, i=0): 
        subfolder = self.subfolders[i]
        path = os.path.join(self.root, subfolder)
            
        folders, files = self.get_folders_files(path)
        print("\nCurrent path depth:", subfolder)
        print("Subfolders:")
        for s in folders: 
            print("\t", s)
    
    def current_path(self):
        return self.subfolders[0]
            
    def number_of_files(self):
        
        lengths = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            _, files = self.get_folders_files(path)
            lengths.append(len(files))
        
        dct = Counter(lengths).most_common()
        most_common = dct[0][0]
        for i, j in dct:
            print(f"\n{j} folders contain {i} files each.")
            
#         differing_subfolders = []
        if len(dct) > 1:
            print("\nFolders with a differing number of files:")
            for subfolder, length in zip(self.subfolders, lengths): 
                if length != most_common: 
#                     differing_subfolders.append(subfolder)
                    print(f"\t{length} files: {subfolder}")
    
        print("\nPath example after current step:")
        print("\t", self.subfolders[0])


## Carotide

In [50]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/afterfifteen/data/recon"
subjects_dir = root + path

carotide = Folder(subjects_dir)
carotide.sample_down()



Current path depth: AF_0292
Subfolders:
	 PWV
	 AF_0292_wip3d05mmcssenseV4
	 AF_0292_PWV_wippwvslice1ccalowsenseV4
	 AF_0292_PWV_wippwvlocalccasenseV4_EP
	 AF_0292_PWV_wippwvslice2icasenseV4
	 af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4


In [4]:
carotide.down(contains=["4dflow"], print_absent=True)


Subfolder equals [],   contains ['4dflow'],   does not contain []

AF_0056 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_27012017_1021264_11_2_wip4dflowktpcaprospective15minV4']

AF_0114 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_20012017_1024356_11_2_wip4dflowktpcaprospective15minV4']

AF_0206 has multiple subfolders with the specified name:
['af_07012017_1502256_11_2_wip4dflowktpcaprospective15minV4', 'seg_4Dflow_PCMRA_vcalukovic']

AF_0160 has multiple subfolders with the specified name:
['af_17032017_0852211_13_2_wip4dflowktpcaprospective15minV4', 'seg_4Dflow_PCMRA_vcalukovic']

AF_0012 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_07012017_1027083_11_2_wip4dflowktpcaprospective15minV4']

AF_0154 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_20012017_0929551_11_2_wip4dflowktpcaprospective15minV4']

AF_0155 has multiple subfo

In [5]:
carotide.up()
carotide.down(contains=["wip4dflow"], print_absent=True)


Path example after step:
	 AF_0292

Subfolder equals [],   contains ['wip4dflow'],   does not contain []

234 of 251 folders contained specified subfolder.

Folders without subfolder
	 AF_0310
	 AF_0352
	 AF_0040
	 AF_0129
	 AF_0233
	 AF_0335
	 AF_0041
	 AF_0018
	 AF_0029
	 AF_0242
	 AF_0003
	 AF_0103
	 AF_0204
	 AF_0358
	 AF_0286
	 AF_0231
	 AF_0280

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4


In [6]:
carotide.down(contains=["dicom"])


Subfolder equals [],   contains ['dicom'],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM


In [7]:
carotide.sample_down()


Path: AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM
Subfolders:
	 PC_MRA
	 M
	 MaxVel
	 FFE
	 tMIP
	 magnitude_systole
	 P
	 S


In [8]:
carotide.down(equals=["pc_mra"])


Subfolder equals ['pc_mra'],   contains [],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/PC_MRA


In [9]:
carotide.number_of_files()


234 folders contain 31 files each.

Path example after current step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/PC_MRA


In [10]:
carotide.up()
carotide.down(equals=["M"])
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/M

234 folders contain 744 files each.

Path example after current step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/M


In [11]:
carotide.up()
carotide.down(contains="mask_left")
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals [],   contains mask_left,   does not contain []

71 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Left

71 folders contain 31 files each.

Path example after current step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Left


In [12]:
carotide.up()
carotide.down(contains="mask_right")
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals [],   contains mask_right,   does not contain []

AF_0056/af_27012017_1021264_11_2_wip4dflowktpcaprospective15minV4/DICOM has multiple subfolders with the specified name:
['Mask_Right_old', 'Mask_Right']

71 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Right

71 folders contain 31 files each.

Path example after current step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Right


## Aorta Volunteers

In [45]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow Volunteers/Aorta"
subjects_dir = root + path

aorta_v = Folder(subjects_dir)
aorta_v.sample_down()



Path: 16-03-18_Sanjay2_kt-pca_done2
Subfolders:
	 te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca
	 te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense


In [25]:
aorta_v.down(contains=["_pca"], print_absent=True)


Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2

Subfolder equals [],   contains ['_pca'],   does not contain []

30 of 34 folders contained specified subfolder.

Folders without subfolder
	 17-03-03, Milo
	 17-11-01_Quintes_student
	 results
	 16-11-18_Male24

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca


In [27]:
aorta_v.down(equals=["dicom"])


Subfolder equals ['dicom'],   contains [],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM


In [ ]:
aorta_v.sample_down()

In [31]:
aorta_v.up()
aorta_v.down(equals=["M"])
aorta_v.number_of_files()


Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/M

28 folders contain 1152 files each.

2 folders contain 1440 files each.

Folders with a differing number of files:
	1440 files: 17-03-15_GJ_Romeijn/4d_22032017_0910594_4_2_wip4dflowktpcaretroV4_PCA/DICOM/M
	1440 files: 17-02-03_Nijveld/4d_03022017_1402026_5_2_wip4dflowktpcaretroV4_PCA/DICOM/M

Path example after current step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/M


In [32]:
aorta_v.up()
aorta_v.down(equals=["mask"])
aorta_v.number_of_files()


Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM

Subfolder equals ['mask'],   contains [],   does not contain []

26 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/mask

24 folders contain 24 files each.

2 folders contain 30 files each.

Folders with a differing number of files:
	30 files: 17-03-15_GJ_Romeijn/4d_22032017_0910594_4_2_wip4dflowktpcaretroV4_PCA/DICOM/mask
	30 files: 17-02-03_Nijveld/4d_03022017_1402026_5_2_wip4dflowktpcaretroV4_PCA/DICOM/mask

Path example after current step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/mask


In [35]:
aorta_v.up()
aorta_v.up()
aorta_v.up()
aorta_v.down(contains=["_sense"])



Subfolder equals [],   contains ['_sense'],   does not contain []

17-02-03_Nijveld has multiple subfolders with the specified name:
['4d_03022017_1402026_5_2_wip4dflowktpcaretroV4_sense', '4d_03022017_1402026_5_2_wip4dflowktpcaretroV4_sense_dont_use']

30 of 34 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense


In [37]:
aorta_v.down(contains="dicom")
aorta_v.sample_down()


Subfolder equals [],   contains dicom,   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense/DICOM


In [39]:
aorta_v.down(contains=["pc_mra"])


Subfolder equals [],   contains ['pc_mra'],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense/DICOM/PC_MRA


In [40]:
aorta_v.number_of_files()


28 folders contain 24 files each.

2 folders contain 30 files each.

Folders with a differing number of files:
	30 files: 17-03-15_GJ_Romeijn/4d_22032017_0910594_4_2_wip4dflowktpcaretroV4_sense/DICOM/PC_MRA
	30 files: 17-02-03_Nijveld/4d_03022017_1402026_5_2_wip4dflowktpcaretroV4_sense/DICOM/PC_MRA

Path example after current step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense/DICOM/PC_MRA


## Aorta BaV

In [62]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow BAV"
subjects_dir = root + path
print(subjects_dir)

aorta_b = Folder(subjects_dir, subject_substr="done")
aorta_b.sample_down(1)


/home/ptenkaate/lood_storage/divi/Projects/4dflow/4D-flow BAV

Current path depth: 16-08-19_119_done1
Subfolders:
	 ko_19082016_1549079_13_2_wip4dflowktpcaretroV4_sense
	 ko_19082016_1549079_13_2_wip4dflowktpcaretroV4_PCA


In [63]:
aorta_b.down(contains=["_PCA"]) 
aorta_b.sample_down()


Subfolder equals [],   contains ['_PCA'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA

Current path depth: 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA
Subfolders:
	 DICOM


In [64]:
aorta_b.down(contains=["dicom"])
aorta_b.sample_down()


Subfolder equals [],   contains ['dicom'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM

Current path depth: 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM
Subfolders:
	 results_heatmap
	 DICOM
	 ROIs
	 results_heatmap_velocity_direction
	 heat_map_higher_lower_masks_direction_AAo_only
	 heat_map_higher_lower_masks_direction
	 Aorta_seg-sense
	 ROIs_BAV_paper
	 results_energy_loss
	 dicom_postprocessed
	 magnitude_systole
	 results_heatmap_wss_direction
	 Aorta


In [65]:
aorta_b.down(equals=["aorta"],print_absent=True)
aorta_b.sample_down()


Subfolder equals ['aorta'],   contains [],   does not contain []

23 of 23 folders contained specified subfolder.

Folders without subfolder

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM/Aorta


In [66]:
aorta_b.sample_down()


Current path depth: 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM/Aorta
Subfolders:
